# Webcam analysis

In [1]:
import os
import json
import time
import pandas as pd

from geopy.geocoders import Nominatim

from util import get_canton_from_latlon, get_img_prefix
from get_data_json import get_cam_loc, get_cam_frame

import requests

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"
}

In [2]:
cam_loc_json = get_cam_loc()
cam_info_json = get_cam_frame()

In [3]:
loc_col_example = list(cam_loc_json.values())[0]
loc_col_example

{'cam': {'longitude': 8.50748,
  'latitude': 47.310213,
  'altitude': 800,
  'angle': 180,
  'north': 30},
 'instances': {'515': {'url': 'https://laf.roundshot.com/',
   'title': 'Adliswil-Felsenegg Luftseilbahn LAF AG'}}}

## Webcam location info

In [4]:
cam_id = list(cam_loc_json.keys())[0]

longitude = list(cam_loc_json.values())[0]['cam']['longitude']
latitude = list(cam_loc_json.values())[0]['cam']['latitude']
altitude = list(cam_loc_json.values())[0]['cam']['altitude']
angle = list(cam_loc_json.values())[0]['cam']['angle']
north = list(cam_loc_json.values())[0]['cam']['north']

# loc_col_example = list(cam_loc_json.values())[0]
instance_id = list(loc_col_example['instances'].keys())[0]

cam_link = list(loc_col_example['instances'].values())[0]['url']

cam_name = list(loc_col_example['instances'].values())[0]['title']

cam_link_ = cam_link.split('/')
cam_abbr = cam_link_[2].split('.')[0]
if len(cam_link_) > 4:
    cam_abbr = cam_abbr + '_' + cam_link_[3]

print('cam_id:', cam_id)
print('longitude:', longitude)
print('latitude:', latitude)
print('altitude:', altitude)
print('angle:', angle)
print('north:', north)
print('longitude:', longitude)
print('instance_id:', instance_id)
print('cam_link:', cam_link)
print('cam_name:', cam_name)
print('cam_abbr:', cam_abbr)

cam_id: 467
longitude: 8.50748
latitude: 47.310213
altitude: 800
angle: 180
north: 30
longitude: 8.50748
instance_id: 515
cam_link: https://laf.roundshot.com/
cam_name: Adliswil-Felsenegg Luftseilbahn LAF AG
cam_abbr: laf


In [5]:
canton1, _ = get_canton_from_latlon(46.07844, 7.77628) # Zermatt
canton2, _ = get_canton_from_latlon(46.507854, 6.498673) # Morges - Port
canton3, _ = get_canton_from_latlon(47.36931, 8.541663) # Zurich Stadthaus
canton1, canton2, canton3

('Valais', 'Vaud', 'Zürich')

## Webcam info

In [6]:
for k in cam_info_json['categories'].keys():
    print(k, len(cam_info_json['categories'][k]))
print('All', len(cam_loc_json.values()))

Tourism associations 63
Weather cams 4
Construction sites 22
Restaurants 5
Hotels 21
Harbors 5
Museums 1
Golf clubs 1
Corporates 28
Schools 5
Villages 28
Airports 10
Hospitals 11
Ski resorts 131
Cities 9
All 345


In [7]:
info_col_names = cam_info_json['categories']['Cities'][0].keys()
info_col_names

dict_keys(['picture', 'link', 'angle', 'name', 'cam_name', 'customer_name'])

In [8]:
category = list(cam_info_json['categories'].keys())[0]
cam_info_example = cam_info_json['categories'][category][0]
thumbnail_url = cam_info_example["picture"]
cam_link = cam_info_example["link"]
angle = cam_info_example["angle"]
name = cam_info_example["name"] # cam_name - customer_name or equal to cam_name/customer_name
cam_name = cam_info_example["cam_name"]
customer_name = cam_info_example["customer_name"]
cam_link_ = cam_link.split('/')
cam_abbr = cam_link_[2].split('.')[0]
if len(cam_link_) > 4:
    cam_abbr = cam_abbr + '_' + cam_link_[3]

print('category:', category)
print('thumbnail_url:', thumbnail_url)
print('cam_link:', cam_link)
print('angle:', angle)
print('name:', name)
print('cam_name:', cam_name)
print('customer_name:', customer_name)
print('cam_abbr:', cam_abbr)

category: Tourism associations
thumbnail_url: https://backend.roundshot.com/cams/122/thumbnail
cam_link: https://aigle.roundshot.com/
angle: 360
name: Aigle Gemeinde
cam_name: Aigle Gemeinde
customer_name: Aigle Gemeinde
cam_abbr: aigle


In [9]:
webcam_stored_url = get_img_prefix(thumbnail_url, headers=headers)

print('webcam_stored_url of', cam_abbr, webcam_stored_url)

webcam_stored_url of aigle ('https://storage2.roundshot.com/53ce585c3fa260.21080772', 200)


### For-loop to construct dataframe and json for each webcam

In [ ]:
request_interval = 0.5

In [12]:
webcams_json_data = dict()
webcam_list = []
for category in cam_info_json['categories'].keys():
    web_instance_ = cam_info_json['categories'][category]
    web_instance_num = len(web_instance_)
    for idx_, cam_info_ in enumerate(web_instance_):
        thumbnail_url_ = cam_info_["picture"]
        cam_link = cam_info_["link"]
        angle = cam_info_["angle"]
        # cam_name - customer_name or equal to cam_name/customer_name
        name = cam_info_["name"]
        cam_name = cam_info_["cam_name"]
        customer_name = cam_info_["customer_name"]
        cam_link_ = cam_link.split('/')
        cam_abbr = cam_link_[2].split('.')[0]
        if len(cam_link_) > 4:
            cam_abbr = cam_abbr + '_' + cam_link_[3]
        stored_url, status_code = get_img_prefix(thumbnail_url_, headers=headers)

        if status_code != 200:
            break

        print(
            '({}: {}/{}) webcam_stored_url of {}: {}'
            .format(
                category,
                idx_+1,
                web_instance_num,
                cam_abbr, 
                stored_url,
                )
            )
        webcams_json_data.update({cam_abbr: stored_url})

        time.sleep(request_interval)

        webcam_dict = {
            'cam_abbr': cam_abbr,
            'cam_name': cam_name,
            'angle': angle,
            'category': category,
            'url': cam_link,
            'stored_url': stored_url,
        }
        webcam_df = pd.DataFrame([webcam_dict])
        webcam_df.index
        webcam_list.append(webcam_df)

        # print('category:', category)
        # print('thumbnail_url:', thumbnail_url)
        # print('cam_link:', cam_link)
        # print('angle:', angle)
        # print('name:', name)
        # print('cam_name:', cam_name)
        # print('customer_name:', customer_name)
        # print('cam_abbr:', cam_abbr)

(Hotels: 1/21) webcam_stored_url of hoteldesbalances: https://storage2.roundshot.com/5b7abf8c98c207.21044267
(Hotels: 2/21) webcam_stored_url of bellevue: https://storage.roundshot.com/53a0470748e768.32694332
(Hotels: 3/21) webcam_stored_url of belvederegrindelwald: https://storage2.roundshot.com/56dffea8826120.23800639
(Hotels: 4/21) webcam_stored_url of belvedere: https://storage2.roundshot.com/53ce5956663437.97348915
(Hotels: 5/21) webcam_stored_url of buergenstock: https://storage.roundshot.com/53a8348147a752.77568194
(Hotels: 6/21) webcam_stored_url of crestapalace: https://storage.roundshot.com/56c729bb5096f4.46498272
(Hotels: 7/21) webcam_stored_url of casaberno: https://storage2.roundshot.com/5b9912a9a0b415.27138200
(Hotels: 8/21) webcam_stored_url of castello-seeschloss: https://storage2.roundshot.com/5beee273957954.04213862
(Hotels: 9/21) webcam_stored_url of hoteledelweiss: https://storage.roundshot.com/53a9860b3a5780.45862775
(Hotels: 10/21) webcam_stored_url of florensreso

In [13]:
all_webcam_df = pd.concat(webcam_list, ignore_index=True)
all_webcam_df.to_csv('all_webcam_info.csv', index=False)

with open(
    os.path.join('webcams.json'), "w"
) as webcams_json_file:
    json.dump(webcams_json_data, webcams_json_file, indent=4, sort_keys=False)

## Misc.

In [10]:
response = requests.get(thumbnail_url, stream=True, headers=headers)

print(response.status_code)
print(response.url)
# https://stackoverflow.com/a/35616488
webcam_stored_url = response.url.rsplit('/', 3)[0]
print('webcam_stored_url of', cam_abbr, webcam_stored_url)

200
https://storage2.roundshot.com/53ce585c3fa260.21080772/2021-12-27/16-53-00/2021-12-27-16-53-00_thumbnail.jpg
webcam_stored_url of aigle https://storage2.roundshot.com/53ce585c3fa260.21080772


In [11]:
# https://github.com/geopy/geopy#geocoding
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"

geolocator = Nominatim(user_agent=headers['User-Agent'])
latitude, longitude = 47.310213, 8.50748 # example
# Felseneggbahn, 8, Felseneggweg, Buchenegg, Stallikon, Bezirk Affoltern, Zürich, 8143, Schweiz/Suisse/Svizzera/Svizra
location = geolocator.reverse(
    query =(latitude, longitude),
    exactly_one = True,
    timeout=10,
)
print(location.address)
# print('address raw:', location.raw)

Felseneggbahn, 8, Felseneggweg, Buchenegg, Stallikon, Bezirk Affoltern, Zürich, 8143, Schweiz/Suisse/Svizzera/Svizra


In [12]:
location.address.replace(" ", "").split(",")

['Felseneggbahn',
 '8',
 'Felseneggweg',
 'Buchenegg',
 'Stallikon',
 'BezirkAffoltern',
 'Zürich',
 '8143',
 'Schweiz/Suisse/Svizzera/Svizra']